In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 55.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=4c06f30ae8efcdc22db496f7f77960cbb79052da0414393648b86d50daccf6df
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
%cd sample_data/

/content/sample_data


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

from pyspark.sql.functions import when
from pyspark.sql.functions import lit

In [ ]:
# Create SparkSession 
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("A5_Sqoop") \
      .getOrCreate() 

##Q1

In [ ]:
df = spark.read.option('header', 'true').csv('problem1_data set.txt')
df.show()

+-------+---+---------+
|   Name|Age|    Place|
+-------+---+---------+
|    ram| 30|bangalore|
|  kapil| 32|hyderabad|
|sathish| 16|  chennai|
|   ravi| 39|bangalore|
| kavita| 12|hyderabad|
|  kavya| 19|   mysore|
+-------+---+---------+



In [ ]:
df.withColumn('18+', when((df.Age > 18), lit('y')).when((df.Age <= 18), lit('n'))).show()

+-------+---+---------+---+
|   Name|Age|    Place|18+|
+-------+---+---------+---+
|    ram| 30|bangalore|  y|
|  kapil| 32|hyderabad|  y|
|sathish| 16|  chennai|  n|
|   ravi| 39|bangalore|  y|
| kavita| 12|hyderabad|  n|
|  kavya| 19|   mysore|  y|
+-------+---+---------+---+



##Q2

In [ ]:
schema = StructType([\
                     StructField('Station_Id', StringType(), True),\
                     StructField('TimeOfReading', IntegerType(), True),\
                     StructField('ReadingType', StringType(), True),\
                     StructField('Temperature', IntegerType(), True),\
                     StructField('N1', StringType(), True),\
                     StructField('N2', StringType(), True),\
                     StructField('E', StringType(), True)\
                     ])

In [ ]:
df = spark.read.csv('problem2_data set.txt', header=True, nullValue='NA', schema=schema)
df.show()

+-----------+-------------+-----------+-----------+----+----+---+
| Station_Id|TimeOfReading|ReadingType|Temperature|  N1|  N2|  E|
+-----------+-------------+-----------+-----------+----+----+---+
|ITE00100554|     18000101|       TMAX|        -75|null|null|  E|
|ITE00100554|     18000101|       TMIN|       -148|null|null|  E|
|GM000010962|     18000101|       PRCP|          0|null|null|  E|
|EZE00100082|     18000101|       TMAX|        -86|null|null|  E|
|EZE00100082|     18000101|       TMIN|       -135|null|null|  E|
|ITE00100554|     18000102|       TMAX|        -60|null|   I|  E|
|ITE00100554|     18000102|       TMIN|       -125|null|null|  E|
|GM000010962|     18000102|       PRCP|          0|null|null|  E|
|EZE00100082|     18000102|       TMAX|        -44|null|null|  E|
|EZE00100082|     18000102|       TMIN|       -130|null|null|  E|
|ITE00100554|     18000103|       TMAX|        -23|null|null|  E|
|ITE00100554|     18000103|       TMIN|        -46|null|   I|  E|
|GM0000109

In [ ]:
df.printSchema()

root
 |-- Station_Id: string (nullable = true)
 |-- TimeOfReading: integer (nullable = true)
 |-- ReadingType: string (nullable = true)
 |-- Temperature: integer (nullable = true)
 |-- N1: string (nullable = true)
 |-- N2: string (nullable = true)
 |-- E: string (nullable = true)



In [ ]:
df.groupBy('Station_Id').min('Temperature').show()

+-----------+----------------+
| Station_Id|min(Temperature)|
+-----------+----------------+
|ITE00100554|            -148|
|GM000010962|               0|
|EZE00100082|            -135|
+-----------+----------------+



##Q3

In [ ]:
sc = spark.sparkContext
search_file = sc.textFile('problem3_data set.txt')
word_counts = search_file.flatMap(lambda line: line.split(' ')).map(lambda word: (word,1)).reduceByKey(lambda x,y: x+y)
df = spark.createDataFrame(word_counts.collect(), schema=['Word', 'Count'])
df.show()

+-------------+-----+
|         Word|Count|
+-------------+-----+
|         best|   29|
|          big|  279|
|         data|  354|
|       hadoop|   96|
|        spark|   42|
|     training|  111|
|        kelly|    2|
| technologies|    3|
|      courses|   53|
|  intellepaat|    1|
|     cloudera|    6|
|      partner|    1|
|        india|   16|
|    microsoft|    3|
|certification|   26|
|           in|  169|
|       online|   58|
|  preparation|    1|
|          BIG|    6|
|         DATA|    7|
+-------------+-----+
only showing top 20 rows



In [ ]:
df.orderBy('Count', ascending=False).show(10)

+---------+-----+
|     Word|Count|
+---------+-----+
|     data|  354|
|      big|  279|
|       in|  169|
| training|  111|
|   course|  104|
|   hadoop|   96|
|   online|   58|
|  courses|   53|
|    spark|   42|
|bangalore|   38|
+---------+-----+
only showing top 10 rows



##Q4

In [ ]:
schema = StructType([\
                     StructField('Customer_Id', IntegerType(), True),\
                     StructField('Product_Id', IntegerType(), True),\
                     StructField('Amount', FloatType(), True)                     
                     ])

In [ ]:
df = spark.read.csv('problem4_data set.txt', header=True, nullValue='NA', schema=schema)
df.show()

+-----------+----------+------+
|Customer_Id|Product_Id|Amount|
+-----------+----------+------+
|         44|      8602| 37.19|
|         35|      5368| 65.89|
|          2|      3391| 40.64|
|         47|      6694| 14.98|
|         29|       680| 13.08|
|         91|      8900| 24.59|
|         70|      3959| 68.68|
|         85|      1733| 28.53|
|         53|      9900| 83.55|
|         14|      1505|  4.32|
|         51|      3378|  19.8|
|         42|      6926| 57.77|
|          2|      4424| 55.77|
|         79|      9291| 33.17|
|         50|      3901| 23.57|
|         20|      6633|  6.49|
|         15|      6148| 65.53|
|         44|      8331| 99.19|
|          5|      3505| 64.18|
|         48|      5539| 32.42|
+-----------+----------+------+
only showing top 20 rows



In [ ]:
df.groupby('Customer_Id').sum('Amount').orderBy('sum(Amount)', ascending=False).show(10)

+-----------+-----------------+
|Customer_Id|      sum(Amount)|
+-----------+-----------------+
|         68|6375.450028181076|
|         73|6206.199985742569|
|         39|6193.109993815422|
|         54|6065.390002984554|
|         71|5995.659991919994|
|          2|5994.589979887009|
|         97|5977.190007060766|
|         46|5963.110011339188|
|         42|5696.840004444122|
|         59|5642.890004396439|
+-----------+-----------------+
only showing top 10 rows



##Q5

In [ ]:
schema = StructType([\
                     StructField('User_Id', IntegerType(), True),\
                     StructField('Movie_Id', IntegerType(), True),\
                     StructField('Rating', IntegerType(), True),\
                     StructField('Timestamp', DoubleType(), True)
                     ])

In [ ]:
df = spark.read.csv('problem5_data set.txt', sep='\t', header=True, nullValue='NA', schema=schema)
df.show()

+-------+--------+------+------------+
|User_Id|Movie_Id|Rating|   Timestamp|
+-------+--------+------+------------+
|    196|     242|     3|8.81250949E8|
|    186|     302|     3|8.91717742E8|
|     22|     377|     1|8.78887116E8|
|    244|      51|     2|8.80606923E8|
|    166|     346|     1|8.86397596E8|
|    298|     474|     4|8.84182806E8|
|    115|     265|     2|8.81171488E8|
|    253|     465|     5|8.91628467E8|
|    305|     451|     3|8.86324817E8|
|      6|      86|     3|8.83603013E8|
|     62|     257|     2|8.79372434E8|
|    286|    1014|     5|8.79781125E8|
|    200|     222|     5| 8.7604234E8|
|    210|      40|     3|8.91035994E8|
|    224|      29|     3|8.88104457E8|
|    303|     785|     3|8.79485318E8|
|    122|     387|     5|8.79270459E8|
|    194|     274|     2|8.79539794E8|
|    291|    1042|     4|8.74834944E8|
|    234|    1184|     2|8.92079237E8|
+-------+--------+------+------------+
only showing top 20 rows



In [ ]:
df.groupBy('Rating').count().show()

+------+-----+
|Rating|count|
+------+-----+
|     1| 6110|
|     3|27145|
|     5|21201|
|     4|34174|
|     2|11370|
+------+-----+

